In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [18]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/본문_맞춤범검사/2020산림복지_본문_맞춤법검사.csv", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['correct_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(164993, 3) Index(['split_str', 'org_idx', 'correct_str'], dtype='object')
164993


In [19]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [20]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        #print(i)
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass
    #display.clear_output(wait=True) # 출력지우기

100%|██████████| 164993/164993 [2:14:25<00:00, 20.46it/s]   


In [21]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['이제', '최선', '봉사', '삶', '산다', '대충', '살', '필리핀', '선교', '시작']
['자다', '하다', '내려놓다', '자다', '하다', '싶다', '이다', '삶다', '자다', '살다']
['부지런하다', '여전하다', '부지런하다', '없다', '재미있다', '넓다', '있다', '좋다', '반갑다', '고맙다']
['는', '을', '을', '고', '고', '를', '으로', '에도', '의', '을']


In [22]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/본문_형태소분석/2020산림복지_본문_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

61641


,명사,빈도
0,등,40659
1,수,32167
2,것,31588
3,및,31168
4,제,30015
...,...,...
61636,황윤석,1
61637,뇌물수수죄,1
61638,갑상선,1
61639,시스플라틴,1


In [23]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/본문_형태소분석/2020산림복지_본문_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1475


,동사,빈도
0,하다,224752
1,되다,46883
2,따르다,15559
3,않다,12770
4,보다,12505
...,...,...
1470,욕먹다,1
1471,총총거리다,1
1472,비틀대다,1
1473,찐득거리다,1


In [24]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/본문_형태소분석/2020산림복지_본문_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1440


,형용사,빈도
0,있다,77262
1,없다,12669
2,이다,11680
3,아니다,9807
4,같다,8223
...,...,...
1435,갸웃하다,1
1436,발광하다,1
1437,우뚝우뚝하다,1
1438,찌뿌드드하다,1


In [25]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/본문_형태소분석/2020산림복지_본문_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

325


,조사,빈도
0,을,171046
1,의,139947
2,에,121102
3,이,101334
4,를,82201
...,...,...
320,까지야,1
321,까지만이라도,1
322,인걸,1
323,에게가,1


In [26]:
df

,split_str,org_idx,correct_str
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...
...,...,...,...
164988,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다,5787,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다.
164989,다음은 2020년 1월 7일부터 20일까지의 일기다,5787,다음은 2020년 1월 7일부터 20일까지의 일기다
164990,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 냥이 ...,5787,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 고양이...
164991,2020년 1월 7일 하루 종일 비가 내린다,5787,2020년 1월 7일 하루 종일 비가 내린다.


In [27]:
len(morphs)

164993

In [30]:
df["morphs"] = morphs
df

,split_str,org_idx,correct_str,morphs
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...,"[이제, 는, 열심히, 부지런히, 최선, 을, 잘, 봉사, 하는, 삶, 을, 내려놓..."
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...,"[필리핀, 단, 기, 선교, 를, 시작, 으로, 2020년, 코로나, 여파, 에도,..."
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...,"[교육, 수료증, 5, 개, 임산물, 가공, 산야, 초, 산림, 경영, 회복, 적,..."
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...,"[뭐, 든, 열심히, 하는, 나, 에게, 맞는, 일, 맞는, 일, 더위, 에도, 재..."
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...,"[오지랖, 이, 넓어, 실수, 도, 많이, 있었지, !, 교육청, 에서, 좋은, 분..."
...,...,...,...,...
164988,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다,5787,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다.,"[더불어, 치병, 중인, 블로그, 이웃, 님, 들과도, 파이팅, 을, 함께, 외쳐,..."
164989,다음은 2020년 1월 7일부터 20일까지의 일기다,5787,다음은 2020년 1월 7일부터 20일까지의 일기다,"[다음, 은, 2020년, 1월, 7일, 부터, 20일, 까지의, 일기, 다]"
164990,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 냥이 ...,5787,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 고양이...,"[올해, 열, 여섯, 살이, 된, 반려, 묘, 냥, 이, 하루, 종일, 토, 하던,..."
164991,2020년 1월 7일 하루 종일 비가 내린다,5787,2020년 1월 7일 하루 종일 비가 내린다.,"[2020년, 1월, 7일, 하루, 종일, 비, 가, 내, 린다, .]"


In [31]:
df.to_csv("../dataset/본문_형태소분석/2020산림복지_본문_형태소분석(형태소만).csv", index=False, encoding='UTF-8')

In [32]:
df["morphs"] = pos_list
df.rename(columns={"morphs":"pos_tagging"}, inplace=True)
df

,split_str,org_idx,correct_str,pos_tagging
0,이제는 열심히 부지런히 최선을 잘 봉사 하는 삶을 내려 놓아도 잘 산다고 해서 대충...,0,이제는 열심히 부지런히 최선을 잘 봉사하는 삶을 내려놓아도 잘 산다고 해서 대충 살...,"[(이제, Noun), (는, Josa), (열심히, Adverb), (부지런하다,..."
1,필리핀단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지런...,0,필리핀 단기선교를 시작으로 2020년 코로나 여파에도 나의 삶은 여전히 열심히 부지...,"[(필리핀, Noun), (단, Modifier), (기, Modifier), (선..."
2,교육 수료증 5개 임산물가공 산야초 산림경영 회복적교사 심리학 국가자격증 1개 떡제...,0,교육 수료증 5개 임산물 가공 산야초 산림경영 회복적 교사 심리학 국가자격증 1개 ...,"[(교육, Noun), (수료증, Noun), (5, Number), (개, Nou..."
3,뭐든 열심히 하는 나에게 맞는일 더위에도 재미 있었고 사람을 멀리하는 나에게 관계를...,0,뭐든 열심히 하는 나에게 맞는일맞는 일더위에도 재미있었고 사람을 멀리하는 나에게 관...,"[(뭐, Noun), (든, Josa), (열심히, Adverb), (하다, Ver..."
4,오지랖이 넓어 실수도 많이 있었지 교육청에서 좋은분들도 만났고 과장님께서는 늘 반갑...,0,오지랖이 넓어 실수도 많이 있었지! 교육청에서 좋은 분들도 만났고 과장님께서는 늘 ...,"[(오지랖, Noun), (이, Josa), (넓다, Adjective), (실수,..."
...,...,...,...,...
164988,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다,5787,더불어 치병 중인 블로그 이웃님들과도 파이팅을 함께 외쳐본다.,"[(더불다, Verb), (치병, Noun), (중인, Noun), (블로그, No..."
164989,다음은 2020년 1월 7일부터 20일까지의 일기다,5787,다음은 2020년 1월 7일부터 20일까지의 일기다,"[(다음, Noun), (은, Josa), (2020년, Number), (1월, ..."
164990,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 냥이 ...,5787,올해 열여섯 살이 된 반려묘 냥이 하루 종일 토하던 날 기운 없이 앉아 있는 고양이...,"[(올해, Noun), (열, Modifier), (여섯, Noun), (살이, N..."
164991,2020년 1월 7일 하루 종일 비가 내린다,5787,2020년 1월 7일 하루 종일 비가 내린다.,"[(2020년, Number), (1월, Number), (7일, Number), ..."


In [33]:
df.to_csv("../dataset/본문_형태소분석/2020산림복지_본문_형태소분석(품사태깅).csv", index=False, encoding='UTF-8')